In [2]:
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np
import pydicom
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import keras
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.optimizers import Adadelta
from keras.backend import get_session

In [3]:
trainpng_path = '/home/hihipch/finger/fingers/train'
trainpng_list = []
for (path, dir, files) in os.walk(trainpng_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.png':
            filePath = "{0}/{1}".format(path, filename)
            # print(filePath)
            trainpng_list.append(filePath)

In [4]:
R5_list=[]
L5_list=[]
for i in trainpng_list:
    if i[-6]+i[-5]=='5R':
        R5_list.append(i)
    if i[-6]+i[-5]=='5L':
        L5_list.append(i)

In [5]:
print(len(R5_list))
print(len(L5_list))

1500
1500


In [6]:
###Create dataframe
# R5=0
# L5=1

R5_df = pd.DataFrame(data={'filename': R5_list, 'label': [0 for i in range(len(R5_list))]})
L5_df = pd.DataFrame(data={'filename': L5_list, 'label': [1 for i in range(len(L5_list))]})

df=R5_df.append(L5_df)
df.columns=['filename', 'label']
df=df.sample(frac=1)
df.reset_index(drop=True,inplace=True)

In [7]:
### One hot encoding of label
y_train = np.array(df['label'].tolist())
y_train = keras.utils.np_utils.to_categorical(y_train)

In [8]:
import imageio

### Resizing DICOM files in the df
x_train=[]

for i in df['filename']:
    pixeldata = imageio.imread(i)
    mean, std = pixeldata.mean(), pixeldata.std()
    pixeldata = (np.clip((pixeldata-mean)/std, -2.0, 2.0)+2.0)/4.0*255
    x_train.append(pixeldata)
x_train=np.array(x_train).reshape(3000, 128, 128, 1).astype('float32')

In [9]:
x_val=x_train[2200:2600]
y_val=y_train[2200:2600]
x_test=x_train[2600:]
y_test=y_train[2600:]

In [10]:
x_train=x_train[:2200]
y_train=y_train[:2200]

In [11]:
def cnn(size):
    KERNEL = (8,8)
    shape = (size[0], size[1], size[2])
    
    # Define Model
    model = Sequential()
    
    # Add Convolution 1
    model.add(Conv2D(filters=8, kernel_size=KERNEL, activation='relu', input_shape=shape))
    
    # Add Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    # Add Convolution 2
    model.add(Conv2D(filters=16, kernel_size=KERNEL, activation='relu', input_shape=shape))
    
    # Add Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    # Add Convolution 3
    model.add(Conv2D(filters=32, kernel_size=KERNEL, activation='relu', input_shape=shape))
    
    # Add Max Pooling
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    
    # Add Output Layer
    model.add(Dropout(0.5))
    model.add(Dense(units=2, activation='softmax'))
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=Adadelta(), metrics=['accuracy'])
    model.summary()
    
    return model

model = cnn(size=x_train.shape[1:])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 121, 121, 8)       520       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 60, 8)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60, 8)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 53, 53, 16)        8208      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 16)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 19, 19, 32)       

In [12]:
hist = model.fit(x_train, y_train, epochs=50, batch_size=16, validation_data=(x_val, y_val))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2200 samples, validate on 400 samples
Epoch 1/50
2200/2200 [==============================] - 3s 1ms/step - loss: 5.7627 - accuracy: 0.5950 - val_loss: 0.0025 - val_accuracy: 1.0000
Epoch 2/50
2200/2200 [==============================] - 1s 604us/step - loss: 0.0317 - accuracy: 0.9927 - val_loss: 2.3974e-05 - val_accuracy: 1.0000
Epoch 3/50
2200/2200 [==============================] - 1s 606us/step - loss: 0.0043 - accuracy: 0.9982 - val_loss: 9.6299e-06 - val_accuracy: 1.0000
Epoch 4/50
2200/2200 [==============================] - 1s 607us/step - loss: 4.9772e-04 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/50
2200/2200 [==============================] - 1s 611us/step - loss: 0.0102 - accuracy: 0.9982 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/50
2200/2200 [==============================] - 1s 604us/step - loss: 0.0020 - accuracy: 0.9995 - val_l

In [13]:
# 테스트 데이터로 평가하기
score = model.evaluate(x_test, y_test, verbose=1)
print('loss=', score[0])
print('accuracy=', score[1])

400/400 [==============================] - 0s 358us/step
loss= 0.0
accuracy= 1.0


In [14]:
### One hot encoding of label
y_train = np.array(df['label'].tolist())
y_train = keras.utils.np_utils.to_categorical(y_train)

In [15]:
import imageio

### Resizing DICOM files in the df
x_train=[]

for i in df['filename']:
    pixeldata = imageio.imread(i)
    mean, std = pixeldata.mean(), pixeldata.std()
    pixeldata = (np.clip((pixeldata-mean)/std, -2.0, 2.0)+2.0)/4.0*255
    pixeldata = pixeldata.astype(np.uint8)
    pixeldata = cv2.cvtColor(pixeldata,cv2.COLOR_GRAY2RGB)
    x_train.append(pixeldata)
x_train=np.array(x_train)

In [16]:
len(df['filename'])

3000

In [17]:
x_val=x_train[2200:2600]
y_val=y_train[2200:2600]
x_test=x_train[2600:]
y_test=y_train[2600:]

In [18]:
x_train=x_train[:2200]
y_train=y_train[:2200]

In [19]:
from keras.applications import ResNet50
from keras.layers import Dense, Input, Activation, Concatenate, Flatten
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adadelta, SGD, Nadam

num_classes=2
input_shape=(128, 128, 3)

resnetmodel = ResNet50(include_top=False,
                            weights='imagenet',
                            input_shape=input_shape, pooling='avg')

x = Dense(num_classes, activation='softmax', name='predictions')(resnetmodel.output)

resnetmodel = Model(resnetmodel.input, x)

sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
resnetmodel.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
hist = resnetmodel.fit(x_train, y_train, epochs=50, batch_size=16, validation_data=(x_val, y_val))

Train on 2200 samples, validate on 400 samples
Epoch 1/50
2200/2200 [==============================] - 44s 20ms/step - loss: 0.0342 - accuracy: 0.9891 - val_loss: 3.3435e-04 - val_accuracy: 1.0000
Epoch 2/50
2200/2200 [==============================] - 22s 10ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 9.7403e-05 - val_accuracy: 1.0000
Epoch 3/50
2200/2200 [==============================] - 22s 10ms/step - loss: 8.7706e-04 - accuracy: 1.0000 - val_loss: 2.0571e-04 - val_accuracy: 1.0000
Epoch 4/50
2200/2200 [==============================] - 22s 10ms/step - loss: 4.3459e-04 - accuracy: 1.0000 - val_loss: 6.8026e-05 - val_accuracy: 1.0000
Epoch 5/50
2200/2200 [==============================] - 22s 10ms/step - loss: 6.1163e-04 - accuracy: 1.0000 - val_loss: 5.1045e-05 - val_accuracy: 1.0000
Epoch 6/50
2200/2200 [==============================] - 22s 10ms/step - loss: 4.2036e-04 - accuracy: 1.0000 - val_loss: 3.5670e-05 - val_accuracy: 1.0000
Epoch 7/50
2200/2200 [===============